# Prepare Sentinel 1 & 2 data for ML model

The goal of this script is to prepare Sentinel 1 and 2 data for a pytorch tranformer model. The data will be saved in `.pt` format, first as raw data and second as normalized data ready to be imported.

First, load appropriate packages and functions.

In [4]:
import pandas as pd
import numpy as np
import os
import torch
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
import re
import sys
from datetime import timedelta

Prepare project paths to access the training data.

In [5]:
proj_paths = ["/Users/gopal/Google Drive/_Research/Research projects/ML/manclassify/app_data/Thailand",
              "/Users/gopalpenny/Library/CloudStorage/GoogleDrive-gopalpenny@gmail.com/My Drive/_Research/Research projects/ML/manclassify/app_data/Thailand"]

proj_path = [path for path in proj_paths if os.path.exists(path)][0]

# ## Prep project path
proj_normpath = os.path.normpath(proj_path)
proj_dirname = proj_normpath.split(os.sep)[-1]
proj_name = re.sub("_classification$","",proj_dirname)
class_path = os.path.join(proj_path, proj_name + "_classification")
ts_path = os.path.join(proj_path, proj_name + "_download_timeseries")

`class_colname` is the name of the column in pt_classes containing the classification data. 

In [6]:
class_colname = 'Subclass2019'

Read point classes data frame, and drop unused columns. Further, create a merged class column where "Other" is used for nonfarm classes

In [7]:
pt_classes = pd.read_csv(os.path.join(class_path, "location_classification.csv"))
pt_classes = pt_classes[['loc_id', 'Class', class_colname]].dropna()

# Create a merged class column where "Other" is used for nonfarm classes
pt_classes['class'] = ['Other' if x!='Farm' else y for x,y in zip(pt_classes['Class'],pt_classes['Subclass2019'])]
pt_classes

,loc_id,Class,Subclass2019,class
0,0,Farm,Plantation,Plantation
1,1,Farm,Crop(Single),Crop(Single)
2,2,Farm,Crop(Single),Crop(Single)
3,3,Farm,Crop(Single),Crop(Single)
4,4,Farm,Plantation,Plantation
...,...,...,...,...
496,496,Farm,Crop(Single),Crop(Single)
497,497,Farm,Crop(Single),Crop(Single)
498,498,Farm,Plantation,Plantation
499,499,Farm,Plantation,Plantation


## Define functions to load original csv files

Do so for both sentinel 1 and sentinel 2

In [8]:
def prep_s2_loc(loc_id, date_range, proj_path):
    ts_path = os.path.join(proj_path,"Thailand_download_timeseries")
    s2_csv_name = f"pt_ts_loc{loc_id}_s2.csv"
    s2_csv_path = os.path.join(ts_path, s2_csv_name)
    s2_ts = pd.read_csv(s2_csv_path)

    # extract dates from image ids
    s2_ts['datestr'] = [re.sub("(^[0-9]+)[a-zA-Z].*","\\1",x) for x in s2_ts.image_id]
    s2_ts['date'] = pd.to_datetime(s2_ts.datestr, format = "%Y%m%d")

    # subset to cloud-free days AND within date_range
    s2_ts = s2_ts[(s2_ts.date >= date_range[0] - timedelta(days = 60)) & 
                  (s2_ts.date <= date_range[1] + timedelta(days = 60)) & 
                  (s2_ts.cloudmask == 0)]

    # calculate day from startday
    date_diff = (s2_ts.date - date_range[0])
    s2_ts['day'] = [x.days for x in date_diff]
    s2_ts['loc_id'] = loc_id

    # select only predictor and position columns, return tensor
    s2_ts_x = s2_ts[['loc_id','day','B8','B4','B3','B2']]
    return s2_ts_x


# %%
def prep_s1_loc(loc_id, date_range, proj_path):
    ts_path = os.path.join(proj_path,"Thailand_download_timeseries")
    
    s1_csv_name = f"pt_ts_loc{loc_id}_s1.csv"
    s1_csv_path = os.path.join(ts_path, s1_csv_name)
    s1_ts = pd.read_csv(s1_csv_path)
    
    # extract dates from image ids
    s1_ts['datestr'] = [re.sub(".*_.*_.*_.*_([0-9]+)T[0-9]+_.*","\\1",x) for x in s1_ts.image_id]
    s1_ts['date'] = pd.to_datetime(s1_ts.datestr, format = "%Y%m%d")
        
    # subset to cloud-free days AND within date_range
    s1_ts = s1_ts[(s1_ts.date >= date_range[0] - timedelta(days = 60)) & 
                  (s1_ts.date <= date_range[1] + timedelta(days = 60))]
    
    s1_ts = s1_ts[['date','HH','VV','VH','HV','angle']]
    
    # calculate day from startday
    date_diff = (s1_ts.date - date_range[0])
    s1_ts['day'] = [x.days for x in date_diff]
    s1_ts['loc_id'] = loc_id
    
    # select only predictor and position columns, return tensor
    s1_ts_x = s1_ts[['loc_id','day','HH','VV','VH','HV','angle']]
    
    return s1_ts_x

## Load Sentinel 1 and 2 data

In [294]:
s1_orig_path = os.path.join(proj_path, 's1_ts_prepped_orig.pt')
if os.path.exists(s1_orig_path):
    loc_s1_ts_tensor = torch.load(s1_orig_path)
    
else:
    # f = IntProgress(min=0, max=pt_classes.shape[0]) # instantiate the bar
    # display(f) # display the bar
    print("prepping s1 tensor file")
    
    s1_ts_list = []
    loc_id_list = []
    for i in np.arange(pt_classes.shape[0]):
        print(".")
        # loc_id = 499
        # print(loc_id)
        loc_id = pt_classes.loc_id.iloc[i]
        
        s1_ts_loc = prep_s1_loc(loc_id, date_range, proj_path)
        s1_ts_loc = s1_ts_loc.groupby(['loc_id','day'],as_index = False).mean()
        s1_ts_tor = torch.tensor(s1_ts_loc.to_numpy())
        s1_ts_list.append(s1_ts_tor)
        
    loc_s1_ts_tensor = torch.cat(s1_ts_list)
    torch.save(loc_s1_ts_tensor, s1_orig_path)
    

s2_orig_path = os.path.join(proj_path, 's2_ts_prepped_orig.pt')

if os.path.exists(s2_orig_path):
    loc_s2_ts_tensor = torch.load(s2_orig_path)
    
else:
    print("prepping s1 tensor file")
    s2_ts_list = []
    loc_id_list = []
    for i in np.arange(pt_classes.shape[0]):
        # loc_id = 499
        print(".")
        loc_id = pt_classes.loc_id.iloc[i]
        # loc_id_list.append(loc_id)
        s2_ts_loc = prep_s2_loc(loc_id, date_range, proj_path)
        s2_ts_loc = s2_ts_loc.groupby(['loc_id','day'],as_index = False).mean()
        s2_ts_tor = torch.tensor(s2_ts_loc.to_numpy())
        s2_ts_list.append(s2_ts_tor)
        
    loc_s2_ts_tensor = torch.cat(s2_ts_list)
    torch.save(loc_s2_ts_tensor, s2_orig_path)

sys.getsizeof(loc_s2_ts_tensor)
sys.getsizeof(loc_s1_ts_tensor)

72

## Limit number of observations

Limit the number of observations for sentinel 1 and sentinel 2 to 64 in a given year. In other words, we can have at most 1 image every ~6 days.

In [295]:
def resample_nearest_days(tensor_orig, days_select, day_col):
    """
    Select rows from tensor orig which are nearest to at least one of the values in days_select
    days_select : tensor
        Vector of evenly-spaced days used to select rows from tensor_orig
    tensor_orig: tensor
        2D tensor with 1 column being the time variable (i.e., days)
    day_col : numeric
        Colum index of tensor_orig containing time variable (days)
    """
    days = tensor_orig[:, day_col]
    
    # tensor_orig
    days_mat = torch.unsqueeze(days, 0).repeat(len(days_select), 1) #.shape
    select_mat = days_select.unsqueeze(1).repeat(1, len(days)) #.shape

    # days_mat #- select_mat
    nearest = torch.argmin(torch.abs(days_mat - select_mat), dim = 1)
    # torch.unsqueeze(torch.from_numpy(days_select),1)
    tensor_resampled = tensor_orig[torch.unique(nearest),:]
    
    return tensor_resampled
    
def resample_id_nearest_days(tensor_full, days_select, id_col, day_col):
    """
    For each id in id_col, use resample_nearest_days to resample days to the closest to days_select
    """
    ts_resampled = torch.zeros(0, tensor_full.shape[1])
    for loc_id in torch.unique(tensor_full[:, id_col]):
        # print(loc_id)
        tensor_orig = tensor_full[tensor_full[:, id_col] == loc_id]
        
        loc_resampled = get_nearest_days(tensor_orig, days_select, day_col = 1)
        ts_resampled = torch.concat((ts_resampled, loc_resampled), dim = 0)#.shape
        
    return ts_resampled

days_select = torch.arange(0, 370, 6)
s1_ts_resampled = resample_id_nearest_days(tensor_full = loc_s1_ts_tensor, 
                                           days_select = days_select, 
                                           id_col = 0, 
                                           day_col = 1)

s1_ts_resampled

s2_ts_resampled = resample_id_nearest_days(tensor_full = loc_s2_ts_tensor, 
                                           days_select = days_select, 
                                           id_col = 0, 
                                           day_col = 1)

s2_ts_resampled

tensor([[   0.,   55., 4320.,  263.,  582.,  304.],
        [   0.,  120., 3896.,  472.,  785.,  560.],
        [   0.,  145., 3809.,  340.,  623.,  346.],
        ...,
        [ 500.,  348., 2590., 1245., 1153.,  844.],
        [ 500.,  363., 3241., 1605., 1565., 1281.],
        [ 500.,  368., 2692., 1226., 1187.,  898.]], dtype=torch.float64)

## Normalize S1 data

In [296]:
# %% Normalize s1 tensor
s1_ts_resampled = s1_ts_resampled[(s1_ts_resampled[:,1] >= 0) & (s1_ts_resampled[:,1] <= 365)]

s1_ts_resampled[torch.isnan(s1_ts_resampled)] = 0

s1_col_means= s1_ts_resampled.mean(dim = 0)#.shape #.unsqueeze(0).repeat(5,1)
s1_col_std= s1_ts_resampled.std(dim = 0)#.shape #.unsqueeze(0).repeat(5,1)
s1_col_means[[0,1]] = 0
s1_col_std[[0]] = 1
s1_col_std[s1_col_std==0] = 1
s1_col_std[[1]] = 365 # normalize days by 365 -- each year ranges from 0 to 1
s1_col_std[[-1]] = 90 # normalize angle by 90 -- angle ranges from 0 to 1

s1_norms = {'s1_col_std' : s1_col_std,
            's1_col_means' : s1_col_means}

s1_ts_resampled_std = s1_col_std.unsqueeze(0).repeat(s1_ts_resampled.shape[0],1)
s1_ts_resampled_mean = s1_col_means.unsqueeze(0).repeat(s1_ts_resampled.shape[0],1)

loc_s1_ts_norm = (s1_ts_resampled - s1_ts_resampled_mean) / s1_ts_resampled_std
print("Normalized sentinel-1 data:", loc_s1_ts_norm.shape)
print("Columns: [loc_id, day_norm, HH, VV, HV, VV, angle]")
print(loc_s1_ts_norm)

Normalized sentinel-1 data: torch.Size([29749, 7])
Columns: [loc_id, day_norm, HH, VV, HV, VV, angle]
tensor([[ 0.0000e+00,  2.4658e-02,  0.0000e+00,  ...,  6.9264e-02,
          0.0000e+00, -1.4274e-02],
        [ 0.0000e+00,  5.7534e-02,  0.0000e+00,  ...,  1.0661e+00,
          0.0000e+00, -1.4248e-02],
        [ 0.0000e+00,  9.0411e-02,  0.0000e+00,  ...,  1.1190e+00,
          0.0000e+00, -1.4197e-02],
        ...,
        [ 5.0000e+02,  9.6438e-01,  0.0000e+00,  ..., -2.8720e-01,
          0.0000e+00,  2.3965e-02],
        [ 5.0000e+02,  9.9178e-01,  0.0000e+00,  ..., -6.9950e-02,
          0.0000e+00,  5.7148e-02],
        [ 5.0000e+02,  9.9726e-01,  0.0000e+00,  ...,  3.1836e-01,
          0.0000e+00,  2.3966e-02]], dtype=torch.float64)


In the above data, the colums refer to:

* loc_id: id of point location
* day_norm: (day since June 1) / 365
* HH, VV, HV, VV: standard normalization
* angle: (deg angle) / 90

Below shows the maximum number of observations for each location

In [297]:
# get max of number of observations per location
# idx = np.arange(loc_ts_norm.shape[0])
loc_id = np.unique(loc_s1_ts_norm[:,0])
num_obs = pd.DataFrame({'loc_id' : np.unique(loc_s1_ts_norm[:,0]).astype('int')})

num_obs['num_obs'] = [loc_s1_ts_norm[loc_s1_ts_norm[:,0]==i,:].shape[0] for i in num_obs['loc_id']]
print("Sentinel 1:")
print("Median number of observations across loc_id's:", num_obs.num_obs.median())
print("Median number of observations across loc_id's", num_obs.num_obs.mean())
max_obs = num_obs.iloc[[num_obs['num_obs'].idxmax()]]
print(f"Max number of observations is {max_obs.num_obs.item()} for loc_id {max_obs.loc_id.item()}")

Sentinel 1:
Median number of observations across loc_id's: 61.0
Median number of observations across loc_id's 59.37924151696607
Max number of observations is 62 for loc_id 1


In [298]:
s1_ts_resampled[s1_ts_resampled[:,0] == 43].shape

torch.Size([61, 7])

In [299]:
# %% Normalize s2 tensor
s2_ts_resampled = s2_ts_resampled[(s2_ts_resampled[:,1] >= 0) & (s2_ts_resampled[:,1] <= 365)]

row_means= s2_ts_resampled.mean(dim = 1)#.shape #.unsqueeze(0).repeat(5,1)
s2_ts_resampled = s2_ts_resampled[~torch.isnan(row_means)]
s2_col_means= s2_ts_resampled.mean(dim = 0)#.shape #.unsqueeze(0).repeat(5,1)
s2_col_std= s2_ts_resampled.std(dim = 0)#.shape #.unsqueeze(0).repeat(5,1)
s2_col_means[[0,1]] = 0
s2_col_std[[0]] = 1
s2_col_std[[1]] = 365 # normalize days by 365 -- each year ranges from 0 to 1

s2_ts_resampled_std = s2_col_std.unsqueeze(0).repeat(s2_ts_resampled.shape[0],1)
s2_ts_resampled_mean = s2_col_means.unsqueeze(0).repeat(s2_ts_resampled.shape[0],1)

loc_s2_ts_norm = (s2_ts_resampled - s2_ts_resampled_mean) / s2_ts_resampled_std

# get max of number of observations per location
# idx = np.arange(loc_ts_norm.shape[0])
loc_id = np.unique(loc_s2_ts_norm[:,0])
num_obs = pd.DataFrame({'loc_id' : np.unique(loc_s2_ts_norm[:,0]).astype('int')})
num_obs['num_obs'] = [loc_s2_ts_norm[loc_s2_ts_norm[:,0]==i,:].shape[0] for i in num_obs['loc_id']]
print("Sentinel 2:")
print("Median number of observations across loc_id's:", num_obs.num_obs.median())
print("Median number of observations across loc_id's", num_obs.num_obs.mean())
max_obs = num_obs.iloc[[num_obs['num_obs'].idxmax()]]
print(f"Max number of observations is {max_obs.num_obs.item()} for loc_id {max_obs.loc_id.item()}")

Sentinel 2:
Median number of observations across loc_id's: 34.0
Median number of observations across loc_id's 33.125748502994014
Max number of observations is 51 for loc_id 96


## Save the model-ready data

The model-ready data is saved as `model_data_s1.pt`, `model_data_s1.pt`, and `model_data_norms.pt`. The first two contain data that is prepared for the torch model. The last one contains the normalization constants for s1 and s2 data.  

In [303]:
norms = {'s1_col_std' : s1_col_std,
         's1_col_means' : s1_col_means,
         's2_col_std' : s2_col_std,
         's2_col_means' : s2_col_means}
# norms
torch.save(norms, os.path.join(proj_path, 'model_data_norms.pt'))
torch.save(loc_s1_ts_norm, os.path.join(proj_path, 'model_data_s1.pt'))
torch.save(loc_s2_ts_norm, os.path.join(proj_path, 'model_data_s2.pt'))

## Prepare class (target) data

In [331]:
# print('All classes')
# print(pt_classes.groupby(['Class','Subclass2019','class']).count())

train_categories = ['Crop(Single)','Crop(Double)','Plantation', 'Other']
train_classes = [0, 1, 2, 3]

classes_df = pd.DataFrame({'class' : train_categories,
              'class_num' : train_classes})

pt_classes_df = pt_classes[pt_classes['class'].isin(classes_df['class'])][['class','loc_id']]
pt_classes_df2 = pd.merge(pt_classes_df, classes_df, on = 'class').sort_values('loc_id')
# 
# torch.from_pandas(pt_classes_df2[['loc_id','class_num']])
model_data_classes = torch.tensor(pt_classes_df2[['loc_id','class_num']].values)

model_data_classes[1:10, :]

tensor([[1, 0],
        [2, 0],
        [3, 0],
        [4, 2],
        [5, 1],
        [6, 3],
        [7, 2],
        [8, 3],
        [9, 0]])

## Save class / target data

The class data contains 2 columns, the first for `loc_id` and the second for the `class_num`. Note that the s1 and s2 datasets have more locations than `model_data_classes.pt`. That is okay, the `model_data_classes.pt` dataset is the master and some loc_id in the s1 and s2 datasets are not needed (e.g., could not clearly identify the class from the app).

In [333]:
torch.save(model_data_classes, os.path.join(proj_path, 'model_data_classes.pt'))

In [334]:
norms_load = torch.load(os.path.join(proj_path, 'model_data_norms.pt'))
norms_load

{'s1_col_std': tensor([  1.0000, 365.0000,   1.0000,   3.4645,   4.1846,   1.0000,  90.0000],
        dtype=torch.float64),
 's1_col_means': tensor([  0.0000,   0.0000,   0.0000, -10.6189, -18.0191,   0.0000,  37.6008],
        dtype=torch.float64),
 's2_col_std': tensor([  1.0000, 365.0000, 712.1756, 604.6329, 361.9943, 342.0948],
        dtype=torch.float64),
 's2_col_means': tensor([   0.0000,    0.0000, 2831.9051, 1125.1194, 1026.9053,  749.2618],
        dtype=torch.float64)}

## Exploratory class analysis

In [346]:
print('All classes')

pt_classes_df2
print(pt_classes.groupby(['Class','Subclass2019','class']).count())

print('\nFinal classes')
print(pt_classes_df2.groupby(['class']).count())

All classes
                                     loc_id
Class     Subclass2019 class               
Farm      Crop(Double) Crop(Double)      68
          Crop(Single) Crop(Single)     278
          Mixed        Mixed              2
          Plantation   Plantation       109
          Unsure       Unsure             4
NonFarm   Forest       Other              3
          Golf         Other              1
          Mixed        Other              4
          Unsure       Other              1
          Urban        Other              1
Uncertain Mixed        Other              9
          Unsure       Other             12
Water     Mixed        Other              5
          Water        Other              4

Final classes
              loc_id  class_num
class                          
Crop(Double)      68         68
Crop(Single)     278        278
Other             40         40
Plantation       109        109


In [349]:
# Example training data
loc_train = pt_classes_ag.groupby('class', group_keys = False).apply(lambda x: x.sample(frac = 0.8))
loc_nontrain = pt_classes_ag[~pt_classes_ag['loc_id'].isin(loc_train.loc_id)]

loc_valid = loc_nontrain.groupby('class', group_keys = False).apply(lambda x: x.sample(frac = 0.5))
loc_test = loc_nontrain[~loc_nontrain['loc_id'].isin(loc_valid.loc_id)]

print('Training (loc_train summary)\n', loc_train.groupby('class').count())
print('\nValidate (loc_test summary)\n', loc_valid.groupby('class').count())
print('\nTesting (loc_test summary)\n', loc_test.groupby('class').count())

Training (loc_train summary)
               loc_id
class               
Crop(Double)      54
Crop(Single)     222
Other             32
Plantation        87

Validate (loc_test summary)
               loc_id
class               
Crop(Double)       7
Crop(Single)      28
Other              4
Plantation        11

Testing (loc_test summary)
               loc_id
class               
Crop(Double)       7
Crop(Single)      28
Other              4
Plantation        11
